<a href="https://colab.research.google.com/github/JONNY-ME/Landslide-Prevention-and-Innovation-challenge/blob/main/Landslide_prediction_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a name = "Libraries"></a>
## 1. Import relevant libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install catboost --quiet

Mounted at /content/drive
     |████████████████████████████████| 76.8 MB 119.6 MB/s 


In [ ]:
# Import libraries
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

<a name = "Load"></a>
## 2. Load files

In [ ]:
# Read files to pandas dataframes
path = '/content/drive/MyDrive/landslide-prevention-and-innovation-challenge/'
Train = pd.read_csv(path+'Train.csv')
Test = pd.read_csv(path+'Test.csv')
sample_submission = pd.read_csv(path+'SampleSubmission.csv')

<a name = "Preview"></a>
## 3. Preview files

In [ ]:
print(Train.shape, Test.shape)

(10864, 227) (5430, 226)


In [ ]:
data = pd.concat([Train.assign(train=1), Test.assign(train=0)])
print(data.shape)
data.head(5)

(16294, 228)


,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,10_elevation,11_elevation,12_elevation,13_elevation,14_elevation,15_elevation,16_elevation,17_elevation,18_elevation,19_elevation,20_elevation,21_elevation,22_elevation,23_elevation,24_elevation,25_elevation,1_slope,2_slope,3_slope,4_slope,5_slope,6_slope,7_slope,8_slope,9_slope,10_slope,11_slope,12_slope,13_slope,14_slope,15_slope,16_slope,17_slope,18_slope,19_slope,20_slope,21_slope,22_slope,23_slope,24_slope,25_slope,1_aspect,2_aspect,3_aspect,4_aspect,5_aspect,6_aspect,7_aspect,8_aspect,9_aspect,10_aspect,11_aspect,12_aspect,13_aspect,14_aspect,15_aspect,16_aspect,17_aspect,18_aspect,19_aspect,20_aspect,21_aspect,22_aspect,23_aspect,24_aspect,25_aspect,1_placurv,2_placurv,3_placurv,4_placurv,5_placurv,6_placurv,7_placurv,8_placurv,9_placurv,10_placurv,11_placurv,12_placurv,13_placurv,14_placurv,15_placurv,16_placurv,17_placurv,18_placurv,19_placurv,20_placurv,21_placurv,22_placurv,23_placurv,24_placurv,25_placurv,1_procurv,2_procurv,3_procurv,4_procurv,5_procurv,6_procurv,7_procurv,8_procurv,9_procurv,10_procurv,11_procurv,12_procurv,13_procurv,14_procurv,15_procurv,16_procurv,17_procurv,18_procurv,19_procurv,20_procurv,21_procurv,22_procurv,23_procurv,24_procurv,25_procurv,1_lsfactor,2_lsfactor,3_lsfactor,4_lsfactor,5_lsfactor,6_lsfactor,7_lsfactor,8_lsfactor,9_lsfactor,10_lsfactor,11_lsfactor,12_lsfactor,13_lsfactor,14_lsfactor,15_lsfactor,16_lsfactor,17_lsfactor,18_lsfactor,19_lsfactor,20_lsfactor,21_lsfactor,22_lsfactor,23_lsfactor,24_lsfactor,25_lsfactor,1_twi,2_twi,3_twi,4_twi,5_twi,6_twi,7_twi,8_twi,9_twi,10_twi,11_twi,12_twi,13_twi,14_twi,15_twi,16_twi,17_twi,18_twi,19_twi,20_twi,21_twi,22_twi,23_twi,24_twi,25_twi,1_geology,2_geology,3_geology,4_geology,5_geology,6_geology,7_geology,8_geology,9_geology,10_geology,11_geology,12_geology,13_geology,14_geology,15_geology,16_geology,17_geology,18_geology,19_geology,20_geology,21_geology,22_geology,23_geology,24_geology,25_geology,1_sdoif,2_sdoif,3_sdoif,4_sdoif,5_sdoif,6_sdoif,7_sdoif,8_sdoif,9_sdoif,10_sdoif,11_sdoif,12_sdoif,13_sdoif,14_sdoif,15_sdoif,16_sdoif,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label,train
0,1,130,129,127,126,123,126,125,124,122,119,122,121,119,117,115,119,117,115,114,112,116,114,113,111,110,35.26439,37.29208,33.85452,35.79576,40.31554,38.87666,39.50971,40.51059,45.83452,45.00000,36.05503,40.51059,44.56372,41.81031,38.87666,33.85452,38.87666,33.85452,33.85452,32.63194,32.63194,30.24626,30.24626,30.24626,30.24626,98.1301,113.1986,116.5650,123.6901,135.0000,97.12502,104.0362,110.5560,119.0546,126.8699,105.9454,110.5560,113.9625,116.5650,119.7449,116.5650,119.7449,116.5650,116.5650,128.6598,128.6598,120.9638,120.9638,120.9638,120.9638,0.038514,0.029463,0.031405,0.025771,0.010453,0.028321,0.027360,0.019831,0.009338,0.008806,0.021451,0.021244,0.017273,0.006708,-0.008671,0.023851,0.012586,-0.004249,0.003646,-0.006809,0.013328,0.005337,0.007039,-0.017753,-0.007777,0.013903,0.014674,0.018249,0.018368,0.003341,0.002023,0.005743,0.002238,-0.003820,-0.003302,-0.002141,0.003585,0.002025,-0.012269,-0.027184,0.000977,-0.012630,-0.031644,-0.025683,-0.037371,-0.018934,-0.021875,-0.023553,-0.048495,-0.039092,8.045186,8.333038,7.819405,8.032228,9.818933,9.295772,9.375107,9.499450,10.922910,11.001900,9.339861,9.883230,11.035840,10.298030,9.618946,8.703197,10.175750,8.651121,8.251134,8.074524,8.519887,7.989215,7.634287,7.804186,7.219216,3.173340,2.961406,3.315935,3.061250,3.246914,3.221661,3.153512,3.048637,2.915506,3.073973,3.765278,3.246664,3.154479,3.237765,3.392537,3.851345,3.673898,3.821337,3.584646,3.734637,4.003083,4.218082,3.990867,4.100921,3.715154,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1.281767,1.281743,1.281708,1.281684,1.281649,1.281780,1.281757,1.281721,1.281698,1.281662,1.281789,1.281765,1.281730,1.281707,1.281671,1.281802,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752

In [ ]:
base_fts = ['elevation', 'lsfactor', 'placurv', 'procurv', 'sdoif', 'slope', 'twi', 'aspect']

for i in base_fts:
  cols = [x for x in data.columns if x.endswith(i)]
  data[f'{i}_mean'] = data[cols].mean(axis=1)
  data[f'{i}_std'] = data[cols].std(axis=1)
  data[f'{i}_max'] = data[cols].max(axis=1)
  data[f'{i}_min'] = data[cols].min(axis=1)
  data[f'{i}_range'] = data[f'{i}_max'] - data[f'{i}_min']
  data[f'{i}_skew'] = data[cols].skew(axis=1)
  data[f'{i}_sum'] = data[cols].sum(axis=1)
  data[f'{i}_sem'] = data[cols].sem(axis=1)
  data[f'{i}_kurtosis'] = data[cols].kurtosis(axis=1)
  data[f'{i}_quantile'] = data[cols].quantile(axis=1)

for i in ['geology']:
  cols = [x for x in data.columns if x.endswith(i)]
  data[f'{i}_mode'] = data[cols].mode(axis=1)[0]
  data[f'{i}_count'] = data[cols].count(axis=1)
  data[f'{i}_mean'] = data[cols].mean(axis=1)

print(data.shape)
data

(16294, 311)


,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,10_elevation,11_elevation,12_elevation,13_elevation,14_elevation,15_elevation,16_elevation,17_elevation,18_elevation,19_elevation,20_elevation,21_elevation,22_elevation,23_elevation,24_elevation,25_elevation,1_slope,2_slope,3_slope,4_slope,5_slope,6_slope,7_slope,8_slope,9_slope,10_slope,11_slope,12_slope,13_slope,14_slope,15_slope,16_slope,17_slope,18_slope,19_slope,20_slope,21_slope,22_slope,23_slope,24_slope,25_slope,1_aspect,2_aspect,3_aspect,4_aspect,5_aspect,6_aspect,7_aspect,8_aspect,9_aspect,10_aspect,11_aspect,12_aspect,13_aspect,14_aspect,15_aspect,16_aspect,17_aspect,18_aspect,19_aspect,20_aspect,21_aspect,22_aspect,23_aspect,24_aspect,25_aspect,1_placurv,2_placurv,3_placurv,4_placurv,5_placurv,6_placurv,7_placurv,8_placurv,9_placurv,10_placurv,11_placurv,12_placurv,13_placurv,14_placurv,15_placurv,16_placurv,17_placurv,18_placurv,19_placurv,20_placurv,21_placurv,22_placurv,23_placurv,24_placurv,25_placurv,1_procurv,2_procurv,3_procurv,4_procurv,5_procurv,6_procurv,7_procurv,8_procurv,9_procurv,10_procurv,11_procurv,12_procurv,13_procurv,14_procurv,15_procurv,16_procurv,17_procurv,18_procurv,19_procurv,20_procurv,21_procurv,22_procurv,23_procurv,24_procurv,25_procurv,1_lsfactor,2_lsfactor,3_lsfactor,4_lsfactor,5_lsfactor,6_lsfactor,7_lsfactor,8_lsfactor,9_lsfactor,10_lsfactor,11_lsfactor,12_lsfactor,13_lsfactor,14_lsfactor,15_lsfactor,16_lsfactor,17_lsfactor,18_lsfactor,19_lsfactor,20_lsfactor,21_lsfactor,22_lsfactor,23_lsfactor,24_lsfactor,25_lsfactor,1_twi,2_twi,3_twi,4_twi,5_twi,6_twi,7_twi,8_twi,9_twi,10_twi,11_twi,12_twi,13_twi,14_twi,15_twi,16_twi,17_twi,18_twi,19_twi,20_twi,21_twi,22_twi,23_twi,24_twi,25_twi,1_geology,2_geology,3_geology,4_geology,5_geology,6_geology,7_geology,8_geology,9_geology,10_geology,11_geology,12_geology,13_geology,14_geology,15_geology,16_geology,17_geology,18_geology,19_geology,20_geology,21_geology,22_geology,23_geology,24_geology,25_geology,1_sdoif,2_sdoif,3_sdoif,4_sdoif,5_sdoif,6_sdoif,7_sdoif,8_sdoif,9_sdoif,10_sdoif,11_sdoif,12_sdoif,13_sdoif,14_sdoif,15_sdoif,16_sdoif,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label,train,elevation_mean,elevation_std,elevation_max,elevation_min,elevation_range,elevation_skew,elevation_sum,elevation_sem,elevation_kurtosis,elevation_quantile,lsfactor_mean,lsfactor_std,lsfactor_max,lsfactor_min,lsfactor_range,lsfactor_skew,lsfactor_sum,lsfactor_sem,lsfactor_kurtosis,lsfactor_quantile,placurv_mean,placurv_std,placurv_max,placurv_min,placurv_range,placurv_skew,placurv_sum,placurv_sem,placurv_kurtosis,placurv_quantile,procurv_mean,procurv_std,procurv_max,procurv_min,procurv_range,procurv_skew,procurv_sum,procurv_sem,procurv_kurtosis,procurv_quantile,sdoif_mean,sdoif_std,sdoif_max,sdoif_min,sdoif_range,sdoif_skew,sdoif_sum,sdoif_sem,sdoif_kurtosis,sdoif_quantile,slope_mean,slope_std,slope_max,slope_min,slope_range,slope_skew,slope_sum,slope_sem,slope_kurtosis,slope_quantile,twi_mean,twi_std,twi_max,twi_min,twi_range,twi_skew,twi_sum,twi_sem,twi_kurtosis,twi_quantile,aspect_mean,aspect_std,aspect_max,aspect_min,aspect_range,aspect_skew,aspect_sum,aspect_sem,aspect_kurtosis,aspect_quantile,geology_mode,geology_count,geology_mean
0,1,130,129,127,126,123,126,125,124,122,119,122,121,119,117,115,119,117,115,114,112,116,114,113,111,110,35.26439,37.29208,33.85452,35.795760,40.31554,38.87666,39.509710,40.510590,45.83452,45.00000,36.055030,40.51059,44.563720,41.81031,38.87666,33.854520,38.87666,33.85452,33.85452,32.631940,32.63194,30.24626,30.24626,30.24626,30.24626,98.13010,113.19860,116.56500,123.6901,135.00000,97.12502,104.036200,110.55600,119.05460,126.86990,105.9454,110.5560,113.96250,116.5650,119.7449,116.56500,119.74490,116.56500,116.565000,128.65980,128.65980,120.96380,120.96380,120.96380,120.96380,0.038514,0.029463,0.031405,0.025771,0.010453,0.028321,0.027360,0.019831,0.009338,0.008806,0.021451,0

In [ ]:
neighbours = ["12", "18", "14", "8", ]

for ft in base_fts:
    for i in neighbours:
        data[f"{ft}_{i}_diff"] = data[f"13_{ft}"] - data[f"{i}_{ft}"]  

for i in base_fts:
  cols = [x for x in data.columns if x.endswith(i) and any(j in x for j in neighbours)]
  data[f'neighbours_{i}_mean'] = data[cols].mean(axis=1)
  data[f'neighbours_{i}_std'] = data[cols].std(axis=1)
  data[f'neighbours_{i}_max'] = data[cols].max(axis=1)
  data[f'neighbours_{i}_min'] = data[cols].min(axis=1)
  data[f'neighbours_{i}_range'] = data[f'neighbours_{i}_max'] - data[f'neighbours_{i}_min']
  data[f'neighbours_{i}_skew'] = data[cols].skew(axis=1)
  data[f'neighbours_{i}_sum'] = data[cols].sum(axis=1)
  data[f'neighbours_{i}_sem'] = data[cols].sem(axis=1)
  data[f'neighbours_{i}_kurtosis'] = data[cols].kurtosis(axis=1)
  data[f'neighbours_{i}_quantile'] = data[cols].quantile(axis=1)

for i in ['geology']:
  cols = [x for x in data.columns if x.endswith(i) and any(j in x for j in neighbours)]
  data[f'neighbours_{i}_mode'] = data[cols].mode(axis=1)[0]
  data[f'neighbours_{i}_count'] = data[cols].count(axis=1)
  data[f'neighbours_{i}_mean'] = data[cols].mean(axis=1)

print(data.shape)

(16294, 426)


In [ ]:
train_df = data[data.train == 1].drop(columns=['train'])
test_df = data[data.train == 0].drop(columns=['train', 'Label'])

train_df.shape, test_df.shape

((10864, 425), (5430, 424))

In [ ]:
main_cols = test_df.columns.difference([
    '17_elevation', '23_elevation', '8_aspect', '17_aspect',
       '21_aspect', '4_procurv', '4_lsfactor', '18_lsfactor',
       '19_lsfactor', '21_twi', '2_geology', '5_geology', '7_geology',
       '12_geology', '13_geology', '14_geology', '19_geology',
       '23_geology', '24_geology', '8_sdoif', '10_sdoif', '12_sdoif',
       '15_sdoif', '17_sdoif', '18_sdoif', '24_sdoif', 'elevation_min',
       'sdoif_sum', 'slope_sum', 'aspect_mean', 'geology_count',
       'geology_12_diff', 'geology_18_diff', 'geology_14_diff',
       'geology_8_diff', 'neighbours_lsfactor_mean',
       'neighbours_lsfactor_sum', 'neighbours_placurv_max',
       'neighbours_placurv_range', 'neighbours_procurv_quantile',
       'neighbours_twi_mean', 'neighbours_twi_quantile',
       'neighbours_aspect_mean', 'neighbours_aspect_max',
       'neighbours_aspect_quantile', 'neighbours_geology_count',
       '7_geology', '11_geology', '15_geology', '20_geology',
       'placurv_sum', 'sdoif_mean', 'sdoif_sum', 'sdoif_quantile',
       'slope_sum', 'aspect_sum',
       'neighbours_elevation_quantile',
       'neighbours_sdoif_mean', 'neighbours_sdoif_max',
       'neighbours_sdoif_sum', 'neighbours_sdoif_quantile',
])
X = train_df[main_cols]
y = train_df.Label
test = test_df[main_cols]

<a name = "Model"></a>
## 9. Model training

In [ ]:
import time 
from sklearn.metrics import f1_score

def squared_mean(results):
  mean_results = np.ones(results[0].shape)
  ln = len(results)
  for i in range(ln):
    mean_results *= results[i]

  return mean_results ** (1/ln)

class My_model:
    def __init__(self, models):
        self.models = models

    def fit_eval_pred(self, X, y, eval_set, val, test):
        results_eval = []
        results_test = []
        for i, model in enumerate(self.models):
            print(f"[Training]............................... Model_{i+1}")
            st = time.time()
            try:
                model.fit(X, y, eval_set=eval_set, verbose=False)
            except Exception as e:
                model.fit(X, y)
            print(f"[Prediction]............................. Model_{i+1}")
            p = model.predict_proba(val)
            print(f"Log loss = {f1_score(eval_set[0][1], np.argmax(p, axis=1))}", end=" "*6)
            results_eval.append(p)
            results_test.append(model.predict_proba(test))
            print(f"Time {time.time() - st :.2f}sec")

        results_eval = squared_mean(results_eval)
        results_test = squared_mean(results_test)

        return results_eval, results_test

In [ ]:
import time
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import imblearn.over_sampling as ovs


seed = 21

def get_models(seed=21):

    cat_params = {"iterations":10000, "learning_rate":0.1, "verbose":200, "random_state":seed,
            'use_best_model':True, 'early_stopping_rounds':200, }

    lgb_params = {'n_estimators':10000, 'random_state':seed, 'early_stopping_round':100, 'verbose':-1, 
            'learning_rate': 0.06918642648131483, 'num_leaves': 20, 'max_depth': 11,
            'min_data_in_leaf': 200, 'max_bin': 200, 'lambda_l1': 10, 'lambda_l2': 20,
            'min_gain_to_split': 0.00011324914950008869, 'bagging_fraction': 0.6000000000000001,
            'bagging_freq': 1, 'feature_fraction': 0.8
            }

    return  [
            CatBoostClassifier(**cat_params),   
            LGBMClassifier(**lgb_params),
    ]

In [ ]:
results = []
ns = 10
tot = 0

skf = StratifiedKFold(n_splits=ns, random_state=seed, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(100*"-", f"\nFold-{fold+1}")

    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    sampler = ovs.SMOTE(random_state=42)
    X_train, y_train = sampler.fit_resample(X_train, y_train)

    main_model = My_model(get_models())
    pred, T_pred = main_model.fit_eval_pred(X_train, y_train, [(X_test, y_test)], X_test, test)

    score = f1_score(y_test, pd.Series([[0, 1][i >= .35] for i in pred[:, 1]]))

    print(f"f1 score = {score}")
    tot += score

    results.append(T_pred)

    print(100*"-", "\n\n\n")

print(f"Average log loss = {tot / ns}")

---------------------------------------------------------------------------------------------------- 
Fold-1
[Training]............................... Model_1
[Prediction]............................. Model_1
Log loss = 0.6904315196998123      Time 34.99sec
[Training]............................... Model_2
[Prediction]............................. Model_2
Log loss = 0.6966292134831461      Time 10.58sec
f1 score = 0.7109634551495017
---------------------------------------------------------------------------------------------------- 



---------------------------------------------------------------------------------------------------- 
Fold-2
[Training]............................... Model_1
[Prediction]............................. Model_1
Log loss = 0.7422303473491774      Time 39.63sec
[Training]............................... Model_2
[Prediction]............................. Model_2
Log loss = 0.7205882352941176      Time 10.43sec
f1 score = 0.7140439932318104
---------------------

In [ ]:
0.7207142105978677
pd.DataFrame({
    "feature":X.columns,
    "values" : main_model.models[0].feature_importances_,
}).sort_values("values", ascending=False)[:20]


,feature,values
288,neighbours_slope_max,2.808347
185,8_slope,2.484862
223,geology_mode,2.414917
194,9_slope,2.228001
255,neighbours_geology_mode,2.127018
350,slope_min,1.855969
209,elevation_12_diff,1.461555
44,15_slope,1.287745
205,aspect_range,1.263864
348,slope_max,1.252105


<a name = "Predictions"></a>
## 10. Test set predictions

In [ ]:
test_pred = pd.Series([[0, 1][i >= .35] for i in np.mean(results, axis=0)[:, 1]])
test_pred.value_counts()

0    3489
1    1941
dtype: int64

<a name = "Submission"></a>
## 11. Creating a submission file

In [ ]:
sub_file = pd.DataFrame({'Sample_ID': Test.Sample_ID, 'Label': test_pred})
sub_file.to_csv('/content/submission1.csv', index = False)
sub_file.head()

,Sample_ID,Label
0,10865,0
1,10866,0
2,10867,0
3,10868,1
4,10869,1
